# Full Data Exploration — Booking (Project Foundation)

## Purpose
This notebook provides a solid understanding of the **Booking dataset** used in the project, and motivates downstream decisions.

We answer:
- What is the entity granularity (is `hotel_id` unique)?
- Which key signals are available and at what coverage?
- How complete are nested fields (arrays/structs), beyond naive NULL checks?
- Which columns are suitable for:
  - **Teacher labeling** (text evidence: `top_reviews + description`)
  - **Student model features** (structured metadata, no review text)



In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import DataFrame
import os

## 0) Load Booking dataset

In [0]:
storage_account = "lab94290"  
acct = storage_account

def set_sas_for_account(acct: str, sas_token: str):
    sas_token = sas_token.lstrip('?')
    spark.conf.set(f"fs.azure.account.auth.type.{acct}.dfs.core.windows.net", "SAS")
    spark.conf.set(f"fs.azure.sas.token.provider.type.{acct}.dfs.core.windows.net",
                   "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
    spark.conf.set(f"fs.azure.sas.fixed.token.{acct}.dfs.core.windows.net", sas_token)
    
# ============================================================
# INSERT SAS TOKEN HERE (provided by course staff)
# Option A (recommended): set env var BOOKING_SAS_TOKEN
# Option B: paste token into booking_sas_token below
# ============================================================
booking_sas_token = os.getenv("BOOKING_SAS_TOKEN", "<INSERT_SAS_TOKEN_HERE>")

if booking_sas_token == "<INSERT_SAS_TOKEN_HERE>" or booking_sas_token.strip() == "":
    raise ValueError("Missing SAS token. Paste it into booking_sas_token or set env var BOOKING_SAS_TOKEN.")


set_sas_for_account(acct, booking_sas_token)

booking_container = "booking"
booking_fname = "booking_1_9.parquet"
booking_path = f"abfss://{booking_container}@{acct}.dfs.core.windows.net/{booking_fname}"

booking_df = spark.read.parquet(booking_path)
n_rows = booking_df.count()
n_cols = len(booking_df.columns)

print("Rows:", n_rows)
print("Cols:", n_cols)
display(booking_df.limit(5))

Rows: 3239391
Cols: 25


availability city coordinates country description fine_print hotel_id house_rules images location managed_by manager_language_spoken manager_score metro_railway_access most_popular_facilities number_of_reviews popular_facilities property_highlights property_information property_surroundings review_score reviews_scores title top_reviews url List(List(1 king bed, null, 2, One-Bedroom Apartment)) Vrindāvan List(27.5737379354334, 77.6546632927292) India Featuring a garden, Mero vrindavan Luxury aavas features accommodations in Vrindāvan. This property offers access to a terrace, free private parking, and free Wifi.

This apartment will provide guests with 1 bedroom, a flat-screen TV, and air conditioning. The apartment also includes 1 bathroom.

Bharatpur Train Station is 31 miles from the apartment, while Mathura Train Station is 9 miles from the property. Agra Airport is 41 miles away. This property does not accommodate bachelor(ette) or similar parties. 12919785 List(List(From 12:00 PM to 11:00 PM You need to let the property know what time you'll be arriving in advance., Check-in), List(From 8:00 AM to 11:00 AM, Check-out), List(Cancellation and prepayment policies vary according to accommodation type. Enter your stay dates and check the conditions of your selected option., Cancellation/ prepayment), List(Child policies Children of all ages are welcome. To see correct prices and occupancy info, add the number and ages of children in your group to your search. Crib and extra bed policies Cribs and extra beds aren't available at this property., Children & Beds), List(There's no age requirement for check-in, No age restriction), List(Smoking is not allowed., Smoking), List(Parties/events are not allowed, Parties), List(Free! Pets are allowed. No extra charges., Pets)) List(https://cf.bstatic.com/xdata/images/hotel/max200/603653035.jpg?k=389f38be85031d2743f99b2fa6dfb9057557c739a921c74399ef5a0ef2d13edd&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653144.jpg?k=f363b0b79f3a6cd34c0deea79fe914db2e9ff4de19718324c9984c473167f21c&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653114.jpg?k=d1db0a18879b81d88ced90184f7baf3d69a655690cb52845fcc66c4734ab1135&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653103.jpg?k=b4196fc9b98950240db78d9648f8c8f88cedad19a0421c8c2c09a056bda0b214&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653086.jpg?k=c947befcb2f56d906e33488fe3bf63dc93a7902d037e2a8e36b8f87dc782261b&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653112.jpg?k=5e4ac49a4cd1ebfceb59314e1c81e80b115250e34a7f8f2784998ea924831857&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653083.jpg?k=ea5c5f07e22411dbe5a8efe3b38d2186907f87c4d6a7b59e4225902920d7e57b&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653080.jpg?k=1a8ad31134e2089042228673eb427c58b040904707917123325d70b8e70bbeb4&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653068.jpg?k=e398c11b34fa5dc518938f033b9234c55b024a0857fa0ff7954399ed1968975f&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653075.jpg?k=d64617957b2a3a2aa8038da09e33c7502bad000c06fea6f2580d4d4ec61931d6&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653077.jpg?k=6e8a8110d71315753c82807a0983eff2cd53c2073abd818f8cec72e646fb2d87&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653088.jpg?k=b2410bbebbf7e7747d63e1eb371676d2b7c142886e49d764bb2eca8ff22ad1b4&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653093.jpg?k=01e8307caea449129657723dd6674cc021f583ec1df15e1bd56aac26ddbee099&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653098.jpg?k=2941324ca574eccb9e3d0eea87e5d46d8151d66b6753ff4fcefd4cdb53203ff7&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653107.jpg?k=2030818f224ee99b97e139e3f0bc79778dd21f32204ae3618fdb13ae7dbac145&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653119.jpg?k=c46bd7884d30ab675cb06330515457f6d56f47bae3ffbc616e449913cae680f6&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653

## 1) Schema overview

In [0]:
print(booking_df._jdf.schema().treeString())

root
 |-- availability: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- bed_configuration: string (nullable = true)
 |    |    |-- max_number_of guests: string (nullable = true)
 |    |    |-- max_number_of_guests: long (nullable = true)
 |    |    |-- room_type: string (nullable = true)
 |-- city: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- lan: double (nullable = true)
 |    |-- lon: double (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- fine_print: string (nullable = true)
 |-- hotel_id: string (nullable = true)
 |-- house_rules: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- rule: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- location: string (nullable = true)
 |-- managed_by: string (nullable =

## 2) Entity granularity: is `hotel_id` unique?

In [0]:
hotel_id_nulls = booking_df.filter(F.col("hotel_id").isNull()).count()
distinct_ids = booking_df.select("hotel_id").distinct().count()
dup_rows = n_rows - distinct_ids

print("hotel_id nulls:", hotel_id_nulls)
print("distinct hotel_id:", distinct_ids)
print("duplicated rows:", dup_rows)
print("duplication rate:", (dup_rows / n_rows) if n_rows else None)


hotel_id nulls: 0
distinct hotel_id: 3239391
duplicated rows: 0
duplication rate: 0.0


## 3) Missingness : strings vs arrays vs structs

In [0]:
def is_missing_expr(path: str, data_type: T.DataType):
    c = F.col(path)

    # STRING
    if isinstance(data_type, T.StringType):
        return c.isNull() | (F.trim(c) == "")
    
    # ARRAY / MAP
    if isinstance(data_type, (T.ArrayType, T.MapType)):
        return c.isNull() | (F.size(c) == 0)
    
    # STRUCT
    if isinstance(data_type, T.StructType):
        child_missing_exprs = []
        for f in data_type.fields:
            child_path = f"{path}.{f.name}"
            child_missing_exprs.append(is_missing_expr(child_path, f.dataType))

        # If struct is NULL → missing
        if not child_missing_exprs:
            return c.isNull()

        # AND all child-missing conditions safely
        all_children_missing = child_missing_exprs[0]
        for expr in child_missing_exprs[1:]:
            all_children_missing = all_children_missing & expr

        return c.isNull() | all_children_missing

    # DEFAULT
    return c.isNull()


## 4) Missingness report for all top-level columns

In [0]:
schema = booking_df.schema

missing_aggs = []
for f in schema.fields:
    miss = is_missing_expr(f.name, f.dataType)
    missing_aggs.append(F.sum(F.when(miss, 1).otherwise(0)).alias(f.name))

missing_counts = booking_df.agg(*missing_aggs)

rows = []
for f in schema.fields:
    rows.append(
        F.struct(
            F.lit(f.name).alias("column"),
            F.lit(f.dataType.simpleString()).alias("dtype"),
            F.col(f.name).cast("long").alias("missing_count"),
            (F.col(f.name) / F.lit(n_rows)).cast("double").alias("missing_rate")
        )
    )

missing_report = (
    missing_counts
    .select(F.explode(F.array(*rows)).alias("r"))
    .select("r.*")
    .orderBy(F.desc("missing_rate"))
)

display(missing_report)


column,dtype,missing_count,missing_rate
reviews_scores,struct,3239391,1.0
managed_by,string,2206939,0.6812820681418205
property_information,string,2172855,0.6707603373597074
manager_score,double,2055721,0.6346010716211782
number_of_reviews,bigint,1691261,0.5220922698124432
review_score,double,1691261,0.5220922698124432
popular_facilities,struct>,1441748,0.4450676068433851
top_reviews,array>,1251691,0.38639701104312507
country,string,1025151,0.316464113162011
location,string,1024963,0.31640607756210964


## 5) Nested-field completeness (inside arrays/structs)

In [0]:
def array_of_struct_field_coverage(df: DataFrame, col_name: str, fields_limit: int = 50) -> DataFrame:
    dt = df.schema[col_name].dataType
    if not isinstance(dt, T.ArrayType) or not isinstance(dt.elementType, T.StructType):
        raise ValueError(f"{col_name} is not array<struct>")

    fields = dt.elementType.fields[:fields_limit]

    array_non_empty_rate = float(
        df.select(
            F.mean(F.when(F.col(col_name).isNull() | (F.size(col_name) == 0), 0).otherwise(1)).alias("r")
        ).first()["r"]
    )

    agg_exprs = []
    field_names = []

    for f in fields:
        # Backtick field name to avoid SQL parse errors
        fname_bt = f"`{f.name}`"
        field_names.append(f.name)

        if isinstance(f.dataType, T.StringType):
            cond_sql = f"exists({col_name}, x -> x.{fname_bt} is not null and trim(x.{fname_bt}) != '')"
        elif isinstance(f.dataType, (T.ArrayType, T.MapType)):
            cond_sql = f"exists({col_name}, x -> x.{fname_bt} is not null and size(x.{fname_bt}) > 0)"
        else:
            cond_sql = f"exists({col_name}, x -> x.{fname_bt} is not null)"

        agg_exprs.append(F.mean(F.when(F.expr(cond_sql), 1).otherwise(0)).alias(f.name))

    field_rates = df.agg(*agg_exprs).first().asDict()

    out_rows = []
    for f in fields:
        out_rows.append((
            col_name,
            f.name,
            f.dataType.simpleString(),
            array_non_empty_rate,
            float(field_rates.get(f.name))
        ))

    return spark.createDataFrame(
        out_rows,
        ["parent_col", "field", "field_dtype", "array_non_empty_rate", "field_present_rate"]
    ).orderBy(F.desc("field_present_rate"))


In [0]:
key_array_struct_cols = [c for c in ["house_rules", "top_reviews", "availability", "property_surroundings"] if c in booking_df.columns]

for c in key_array_struct_cols:
    dt = booking_df.schema[c].dataType
    if isinstance(dt, T.ArrayType) and isinstance(dt.elementType, T.StructType):
        print(f"\n=== Nested coverage for {c} ===")
        display(array_of_struct_field_coverage(booking_df, c))
    else:
        print(f"Skipping {c}: not array<struct>")



=== Nested coverage for house_rules ===


parent_col,field,field_dtype,array_non_empty_rate,field_present_rate
house_rules,rule,string,0.9962168815064313,0.9962168815064313
house_rules,description,string,0.9962168815064313,0.9962168815064313



=== Nested coverage for top_reviews ===


parent_col,field,field_dtype,array_non_empty_rate,field_present_rate
top_reviews,location,string,0.6136029889568749,0.6136029889568749
top_reviews,review,string,0.6136029889568749,0.6136029889568749
top_reviews,reviewer_name,string,0.6136029889568749,0.6119338480597124



=== Nested coverage for availability ===


parent_col,field,field_dtype,array_non_empty_rate,field_present_rate
availability,room_type,string,0.9824037913299135,0.9824025565299157
availability,max_number_of_guests,bigint,0.9824037913299135,0.9822380194301954
availability,bed_configuration,string,0.9824037913299135,0.9606142018669559
availability,max_number_of guests,string,0.9824037913299135,0.0



=== Nested coverage for property_surroundings ===


parent_col,field,field_dtype,array_non_empty_rate,field_present_rate
property_surroundings,location_name,string,0.778991174575715,0.778991174575715
property_surroundings,distance,double,0.778991174575715,0.778991174575715
property_surroundings,distance_unit,string,0.778991174575715,0.778991174575715


## 6) Struct completeness (example: coordinates)

In [0]:
def struct_field_missingness(df: DataFrame, struct_col: str) -> DataFrame:
    st = df.schema[struct_col].dataType
    if not isinstance(st, T.StructType):
        raise ValueError(f"{struct_col} is not struct")

    exprs = []
    for f in st.fields:
        miss = is_missing_expr(f"{struct_col}.{f.name}", f.dataType)
        exprs.append(F.mean(F.when(miss, 1).otherwise(0)).alias(f"{f.name}_missing_rate"))

    return df.select(*exprs)

if "coordinates" in booking_df.columns and isinstance(booking_df.schema["coordinates"].dataType, T.StructType):
    display(struct_field_missingness(booking_df, "coordinates"))
else:
    print("No usable 'coordinates' struct column found.")


lan_missing_rate,lon_missing_rate
9.545003983773493E-4,9.545003983773493E-4


## 7) Review signals availability (Teacher feasibility)

In [0]:
def col_coverage(df: DataFrame, col_name: str) -> float:
    dt = df.schema[col_name].dataType
    miss = is_missing_expr(col_name, dt)
    return float(df.select(F.mean(F.when(miss, 1).otherwise(0)).alias("mr")).first()["mr"])


In [0]:
review_cols = ["review_score", "number_of_reviews", "description", "top_reviews"]
summary = []

for c in review_cols:
    if c in booking_df.columns:
        mr = col_coverage(booking_df, c)
        summary.append((c, booking_df.schema[c].dataType.simpleString(), 1.0 - mr, mr))

review_summary_df = spark.createDataFrame(summary, ["column", "dtype", "present_rate", "missing_rate"]) \
                         .orderBy(F.desc("present_rate"))

display(review_summary_df)


column,dtype,present_rate,missing_rate
description,string,0.9999987652000021,1.23479999790084E-6
top_reviews,array>,0.6136029889568749,0.38639701104312507
number_of_reviews,bigint,0.47790773018755683,0.5220922698124432
review_score,double,0.47790773018755683,0.5220922698124432


## 8) Actionable conclusions (tie EDA → pipeline decisions)

This section turns EDA into explicit, motivated decisions used later in the project.

### Decisions motivated by this notebook
1. **Granularity:** Each row is one listing (hotel) because `hotel_id` is unique.
2. **Spatial enrichment is feasible:** Coordinate coverage is high enough to support external POI enrichment.
3. **Weak supervision is feasible:** Text evidence exists at scale (`description` + `top_reviews` coverage).
4. **Student model avoids review text:** Review-related numerical fields are sparse; relying on them would reduce coverage and bias the system toward popular listings.
5. **Nested fields are used selectively:** We use nested arrays/structs only where internal field coverage is meaningful (see nested coverage tables).
